In [1]:
import pandas as pd
from keras.layers import Dense, Dropout, Activation, Input
from keras.models import Model 

In [2]:
base=pd.read_csv('games.csv')
base=base.drop('Other_Sales',axis=1)
base=base.drop('Global_Sales',axis=1)
base=base.drop('Developer',axis=1) 

In [3]:
base=base.dropna(axis=0)
base=base.loc[base['NA_Sales']>1]
base=base.loc[base['EU_Sales']>1]

In [4]:
base['Name'].value_counts()
nome_jogos=base.Name
base=base.drop('Name',axis=1)

In [5]:
previsores=base.iloc[:,[0,1,2,3,7,8,9,10,11]].values
venda_na=base.iloc[:,4].values
venda_eu=base.iloc[:,5].values
venda_jp=base.iloc[:,6].values

In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_previsores = LabelEncoder()
previsores[:, 0] = labelencoder_previsores.fit_transform(previsores[:, 0])
previsores[:, 2] = labelencoder_previsores.fit_transform(previsores[:, 2])
previsores[:, 3] = labelencoder_previsores.fit_transform(previsores[:, 3])
previsores[:, 8] = labelencoder_previsores.fit_transform(previsores[:, 8])


In [7]:
categorical_columns = [0, 2, 3, 8]
onehotencoder = ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(), categorical_columns)],
    remainder='passthrough')
previsores = ct.fit_transform(previsores).toarray()

In [8]:
camada_entrada=Input(shape=(61,))
camada_oculta1=Dense(units=32,activation='sigmoid')(camada_entrada)
camada_oculta2=Dense(units=32,activation='sigmoid')(camada_oculta1)
camada_saida1=Dense(units=1,activation='linear')(camada_oculta2)
camada_saida2=Dense(units=1,activation='linear')(camada_oculta2)
camada_saida3=Dense(units=1,activation='linear')(camada_oculta2)

In [9]:
regressor=Model(inputs=camada_entrada,
                outputs=[camada_saida1,camada_saida2,camada_saida3])

In [10]:
regressor.compile(optimizer='adam', loss='mse')

In [11]:
regressor.fit(previsores,[venda_na,venda_eu,venda_jp], epochs=5000, batch_size=100)

Epoch 1/5000


In [ ]:
previsao_na,previsao_eu,previsao_jp = regressor.predict(previsores)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
